## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2023-05-23 23:12:19.666667: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
apptype_vcounts = application_df['APPLICATION_TYPE'].value_counts()
apptype_vcounts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(apptype_vcounts[apptype_vcounts<160].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
binning = application_df['CLASSIFICATION'].value_counts()
binning

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
binning[binning>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(binning[binning<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, dtype=float)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df.copy()
X.drop('IS_SUCCESSFUL', axis=1)
#X.head()

y = application_df['IS_SUCCESSFUL'].values.reshape(-1,1)
#y[:5]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=6, activation='relu', input_dim=len( X_train_scaled[0])))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6)                 270       
                                                                 
 dense_4 (Dense)             (None, 6)                 42        
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 319
Trainable params: 319
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# Train the model
fit_model = model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.2771 - accuracy: 0.8818
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.0316 - accuracy: 0.9966
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.0107 - accuracy: 0.9988
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.0047 - accuracy: 0.9994
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.0027 - accuracy: 0.9998
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.0017 - accuracy: 0.9999
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.0013 - accuracy: 0.9999
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.0011 - accuracy: 0.9998
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 9.5220e-04 - accuracy: 0.9999
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 8.9316e-04 

804/804 [==============================] - 1s 2ms/step - loss: 2.4463e-04 - accuracy: 0.9999
Epoch 79/100
804/804 [==============================] - 1s 2ms/step - loss: 2.4412e-04 - accuracy: 0.9999
Epoch 80/100
804/804 [==============================] - 1s 2ms/step - loss: 2.4269e-04 - accuracy: 0.9999
Epoch 81/100
804/804 [==============================] - 1s 2ms/step - loss: 2.4511e-04 - accuracy: 0.9999
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 2.4490e-04 - accuracy: 0.9999
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 2.4488e-04 - accuracy: 0.9999
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 2.4501e-04 - accuracy: 0.9999
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 2.4465e-04 - accuracy: 0.9999
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 2.4315e-04 - accuracy: 0.9999
Epoch 87/100
804/804 [==============================] - 1s 

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.5947e-04 - accuracy: 1.0000 - 400ms/epoch - 1ms/step
Loss: 0.00015946992789395154, Accuracy: 1.0


In [22]:
# Export our model to HDF5 file
model.save("AlphabetSoupCharity.h5")

from google.colab import files
files.download('AlphabetSoupCharity.h5')

ModuleNotFoundError: No module named 'google.colab'